In [1]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
spark

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Зальем данные на spark

In [4]:
from pyspark.sql.types import *

In [5]:
!hdfs dfs -head /labs/lab06data/ml-100k/u.user

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703
11|39|F|other|30329
12|28|F|other|06405
13|47|M|educator|29206
14|45|M|scientist|55106
15|49|F|educator|97301
16|21|M|entertainment|10309
17|30|M|programmer|06355
18|35|F|other|37212
19|40|M|librarian|02138
20|42|F|homemaker|95660
21|26|M|writer|30068
22|25|M|writer|40206
23|30|F|artist|48197
24|21|F|artist|94533
25|39|M|engineer|55107
26|49|M|engineer|21044
27|40|F|librarian|30030
28|32|M|writer|55369
29|41|M|programmer|94043
30|7|M|student|55436
31|24|M|artist|10003
32|28|F|student|78741
33|23|M|student|27510
34|38|F|administrator|42141
35|20|F|homemaker|42459
36|19|F|student|93117
37|23|M|student|55105
38|28|F|other|54467
39|41|M|entertainment|01040
40|38|M|scientist|27514
41|33|M|engineer|80525
42|30|M|administrator|17870
43|29|F|librarian|20854
44|26|M|technic

In [6]:
schema_user = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("age", IntegerType()),
    StructField("gender", StringType()),
    StructField("profession", StringType()),
    StructField("timesamp", IntegerType())
])

In [7]:
users = spark.read.csv("/labs/lab06data/ml-100k/u.user", schema=schema_user, sep="|")

In [8]:
users.count()

943

In [9]:
!hdfs dfs -tail /labs/lab06data/ml-100k/u.item

ct?Sunchaser,%20The%20(1996)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
1676|War at Home, The (1996)|01-Jan-1996||http://us.imdb.com/M/title-exact?War%20at%20Home%2C%20The%20%281996%29|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
1677|Sweet Nothing (1995)|20-Sep-1996||http://us.imdb.com/M/title-exact?Sweet%20Nothing%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
1678|Mat' i syn (1997)|06-Feb-1998||http://us.imdb.com/M/title-exact?Mat%27+i+syn+(1997)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
1679|B. Monkey (1998)|06-Feb-1998||http://us.imdb.com/M/title-exact?B%2E+Monkey+(1998)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|1|0|0
1680|Sliding Doors (1998)|01-Jan-1998||http://us.imdb.com/Title?Sliding+Doors+(1998)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|1|0|0|0|0
1681|You So Crazy (1994)|01-Jan-1994||http://us.imdb.com/M/title-exact?You%20So%20Crazy%20(1994)|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0
1682|Scream of Stone (Schrei aus Stein) (1991)|08-Mar-1996||http://us.imdb.com/M/title-exact?Schrei%20aus%20Stein%20(1991)|0|0|0|0|0|0|0|

In [10]:
schema_data = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("rating", IntegerType()),
    StructField("timestamp", IntegerType())
])

In [11]:
data = spark.read.csv("/labs/lab06data/ml-100k/u.data", schema=schema_data, sep="\t")

In [12]:
data.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [18]:
data.count()

100000

In [13]:
schema_item = StructType(fields=[
    StructField("item_id", IntegerType()),
    StructField("name", StringType()),
    StructField("timestamp", StringType()),
    StructField("emppy", StringType()),         
    StructField("url", StringType()),
    StructField("id1", IntegerType()), 
    StructField("id2", IntegerType()),
    StructField("id3", IntegerType()), 
    StructField("id4", IntegerType()),
    StructField("id5", IntegerType()), 
    StructField("id6", IntegerType()),
    StructField("id7", IntegerType()), 
    StructField("id8", IntegerType()),
    StructField("id9", IntegerType()), 
    StructField("id10", IntegerType()),
    StructField("id11", IntegerType()), 
    StructField("id12", IntegerType()),
    StructField("id13", IntegerType()), 
    StructField("id14", IntegerType()),
    StructField("id15", IntegerType()), 
    StructField("id16", IntegerType()),
    StructField("id17", IntegerType()),
    StructField("id18", IntegerType()), 
    StructField("id19", IntegerType())])

In [14]:
item = spark.read\
          .schema(schema_item)\
          .format("csv")\
          .option("sep", "|")\
          .load("/labs/lab06data/ml-100k/u.item")

In [15]:
item.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- emppy: string (nullable = true)
 |-- url: string (nullable = true)
 |-- id1: integer (nullable = true)
 |-- id2: integer (nullable = true)
 |-- id3: integer (nullable = true)
 |-- id4: integer (nullable = true)
 |-- id5: integer (nullable = true)
 |-- id6: integer (nullable = true)
 |-- id7: integer (nullable = true)
 |-- id8: integer (nullable = true)
 |-- id9: integer (nullable = true)
 |-- id10: integer (nullable = true)
 |-- id11: integer (nullable = true)
 |-- id12: integer (nullable = true)
 |-- id13: integer (nullable = true)
 |-- id14: integer (nullable = true)
 |-- id15: integer (nullable = true)
 |-- id16: integer (nullable = true)
 |-- id17: integer (nullable = true)
 |-- id18: integer (nullable = true)
 |-- id19: integer (nullable = true)



In [16]:
item.select("item_id", "name", "timestamp", "url").where("item_id = 1422").show(3,False, True)

-RECORD 0-----------------------------------------------------
 item_id   | 1422                                             
 name      | Suture (1993)                                    
 timestamp | 01-Jan-1993                                      
 url       | http://us.imdb.com/M/title-exact?Suture%20(1993) 



# Лаб8

In [ ]:
import pyspark.sql.functions as f

In [23]:
data.registerTempTable("data_table")

Считаем средний реитинг для каждого пользователя.

Часть 2. User-user CF:

1. Для каждого пользователя найдите его средний рейтинг (сумма рейтингов пользователя/количество рейтингов пользователя). Здесь Ia — множество фильмов, по которым у пользователя есть рейтинги rui. Здесь и далее |Ia| обозначает количество элементов в множестве Ia.


In [24]:
query_avg_user = """
SELECT t.user_id,
       sum(rating) as sum_rating, 
       count(item_id) as count_rating,
       sum(rating)/count(item_id) as avr_rating
FROM   data_table t
group by t.user_id
"""       

In [27]:
data_user = spark.sql(query_avg_user).repartition(4).cache()

In [28]:
data_user.where("user_id in (29, 758)").show(5)

+-------+----------+------------+------------------+
|user_id|sum_rating|count_rating|        avr_rating|
+-------+----------+------------+------------------+
|     29|       124|          34|3.6470588235294117|
|    758|      1384|         357| 3.876750700280112|
+-------+----------+------------+------------------+



Фильмы с нужным вариантом.

In [30]:
item_758 = data.select("item_id").where("user_id=758")

In [31]:
data_user.registerTempTable("data_user_table")

2. Посчитайте меру близости Пирсона выданного вам пользователя со всеми остальными пользователями. Обратите внимание, что корреляция Пирсона считается только на пересечении, то есть вклад вносят только фильмы, оцененные совместно (Ia, Iu — множества оцененных пользователями a и u фильмов). Корреляция с константой (ситуация, когда у пользователя все оценки одинаковые) равна нулю. 

В запросе это p

3.Посчитайте поправочный коэффициент для корреляции Пирсона на нехватку данных.

В запросе это koef

4.Найдите 30 ближайших пользователей-соседей данного пользователя (pearson_neighbours), используя поправленную корреляцию Пирсона

В запросе Пирсон - это rr
Количество мерим row_number() по убыванию корреляции

Данные в запросе считаются только по общим фильмам у нашего пользователя и всех остальных.

In [59]:
query1 = """
select user_id, rr, koef,p, row_number() over(ORDER BY rr desc) num
from  (
SELECT t2.user_id,
       (case when count(t2.item_id)/50 < 1 then count(t2.item_id)/50 else 1 end) *
       sum((t1.rating-dt1.avr_rating)*(t2.rating-dt2.avr_rating))/
        ((sqrt(sum((t1.rating-dt1.avr_rating)*(t1.rating-dt1.avr_rating))))*
         (sqrt(sum((t2.rating-dt2.avr_rating)*(t2.rating-dt2.avr_rating))))) as rr,
         
       case when count(t2.item_id)/50 < 1 then count(t2.item_id)/50 else 1 end as koef,
       
       sum((t1.rating-dt1.avr_rating)*(t2.rating-dt2.avr_rating))/
       ((sqrt(sum((t1.rating-dt1.avr_rating)*(t1.rating-dt1.avr_rating))))*
        (sqrt(sum((t2.rating-dt2.avr_rating)*(t2.rating-dt2.avr_rating))))) as p
        
FROM   data_table t1 
join   data_table t2 on t1.item_id = t2.item_id
join   data_user_table dt1 on dt1.user_id = t1.user_id
join   data_user_table dt2 on dt2.user_id = t2.user_id
where t1.user_id = 758 
AND   t2.user_id <> 758
group by t2.user_id)
"""       

In [60]:
neighbours = spark.sql(query1).repartition(4).cache()

In [61]:
neighbours_list = spark.sql(query1).take(30)

In [62]:
neighbours_list

[Row(user_id=381, rr=0.5853978722663211, koef=1.0, p=0.5853978722663211, num=1),
 Row(user_id=741, rr=0.5459853163084785, koef=1.0, p=0.5459853163084785, num=2),
 Row(user_id=303, rr=0.5258646182437792, koef=1.0, p=0.5258646182437792, num=3),
 Row(user_id=892, rr=0.5130745966320297, koef=1.0, p=0.5130745966320297, num=4),
 Row(user_id=343, rr=0.5000162518453979, koef=1.0, p=0.5000162518453979, num=5),
 Row(user_id=640, rr=0.4833442802540537, koef=1.0, p=0.4833442802540537, num=6),
 Row(user_id=77, rr=0.47394946232087243, koef=0.94, p=0.5042015556605026, num=7),
 Row(user_id=268, rr=0.47382412556712833, koef=1.0, p=0.47382412556712833, num=8),
 Row(user_id=397, rr=0.4638853118060365, koef=1.0, p=0.4638853118060365, num=9),
 Row(user_id=267, rr=0.4626720401786975, koef=1.0, p=0.4626720401786975, num=10),
 Row(user_id=854, rr=0.4617060253043892, koef=1.0, p=0.4617060253043892, num=11),
 Row(user_id=890, rr=0.46090113148624506, koef=1.0, p=0.46090113148624506, num=12),
 Row(user_id=543, rr

In [41]:
neighbours.show(5)

+-------+------------------+----+------------------+
|user_id|                rr|koef|                 p|
+-------+------------------+----+------------------+
|    294|0.2038110594679973| 1.0|0.2038110594679973|
|    741|0.5459853163084785| 1.0|0.5459853163084785|
|    276|0.3762606341923976| 1.0|0.3762606341923976|
|    334|0.3716597603142623| 1.0|0.3716597603142623|
|    115|0.3546515367759388| 1.0|0.3546515367759388|
+-------+------------------+----+------------------+
only showing top 5 rows



5. Для всех фильмов найдите прогноз оценки по формуле ниже. Здесь N(a) — множество пользователей-соседей, s(a,u) — мера близости пользователей из предыдущих пунктов, |s(a,u)| — модуль меры близости.

In [42]:
neighbours.registerTempTable("neighbours_table")

In [43]:
item_758.registerTempTable("item_758_table") 

In [65]:
query2 = """
select d.item_id,
       3.876750700280112 + 
       sum(n.rr*(d.rating-u.avr_rating))/sum(abs(n.rr)) as rating_new
from   data_table d 
join   neighbours_table n on d.user_id = n.user_id
join   data_user_table u on u.user_id = d.user_id
join   item_758_table i on d.item_id <> i.item_id
where  n.num <31
group by d.item_id
order by 2 desc,1
"""

In [66]:
item_out = spark.sql(query2).take(10)

In [67]:
item_out

[Row(item_id=793, rating_new=5.423219101767111),
 Row(item_id=923, rating_new=5.285617695353995),
 Row(item_id=1449, rating_new=5.285617695353995),
 Row(item_id=1512, rating_new=5.285617695353995),
 Row(item_id=1558, rating_new=5.285617695353995),
 Row(item_id=119, rating_new=5.208329647648536),
 Row(item_id=661, rating_new=5.17368001037895),
 Row(item_id=60, rating_new=5.1710065708874655),
 Row(item_id=611, rating_new=5.170868347338941),
 Row(item_id=741, rating_new=5.1408659823158285)]

In [68]:
list_out_n = [x['user_id'] for x in neighbours_list]

In [69]:
pearson_top10 = [x['item_id'] for x in item_out]

In [70]:
pearson_top10

[793, 923, 1449, 1512, 1558, 119, 661, 60, 611, 741]

In [71]:
list_out_n

[381,
 741,
 303,
 892,
 343,
 640,
 77,
 268,
 397,
 267,
 854,
 890,
 543,
 344,
 121,
 592,
 409,
 776,
 757,
 407,
 118,
 933,
 870,
 293,
 216,
 538,
 345,
 643,
 246,
 457]

Часть1. Основные характеристики данных MovieLens:

1. Найдите количество всех пользователей и количество всех фильмов в данных (общее число пользователей и фильмов в датасете).

In [72]:
data_user_distict = data[["user_id"]].distinct()

In [73]:
data_user_distict.count()

943

In [74]:
data_item_distict = data[["item_id"]].distinct()

In [76]:
data_item_distict.count()

1682

2.Сколько пользователь в среднем ставит рейтингов? Подсчитать количество рейтингов / количество пользователей. 

In [87]:
average_user_ratings = round(data.count()/data_user_distict.count(),4)

In [88]:
average_user_ratings

106.0445

3. Сколько фильм в среднем имеет рейтингов? Подсчитатьколичество рейтингов / количество фильмов. Поле 

In [89]:
average_film_ratings=round(data.count()/data_item_distict.count(),4)

In [90]:
average_film_ratings

59.453

4. Найдите процент заполненных ячеек в данных: количество рейтингов / (количество пользователей * количество фильмов).

In [82]:
completeness = round(data.count()/(data_user_distict.count()*data_item_distict.count()),4)

In [83]:
completeness

0.063

Сохраняем в json

In [84]:
dict_out1 = { "average_film_ratings": average_film_ratings,
    "average_user_ratings": average_user_ratings,
    "completeness": completeness,
    "pearson_neighbours": list_out_n,
    "pearson_top10": pearson_top10
     
}

In [85]:
dict_out1

{'average_film_ratings': 59.453,
 'average_user_ratings': 106.0445,
 'completeness': 0.063,
 'pearson_neighbours': [381,
  741,
  303,
  892,
  343,
  640,
  77,
  268,
  397,
  267,
  854,
  890,
  543,
  344,
  121,
  592,
  409,
  776,
  757,
  407,
  118,
  933,
  870,
  293,
  216,
  538,
  345,
  643,
  246,
  457],
 'pearson_top10': [793, 923, 1449, 1512, 1558, 119, 661, 60, 611, 741]}

In [86]:
import json
with open('lab08.json', 'w') as fout:
    fout.write(json.dumps(dict_out1))

# Ачивка

# Часть 3. Базовые предикторы:

1. Глобальное среднее 𝞵 (average_rating) по всему датасету. Сумма всех оценок по всем фильмам / Количество всех оценок по всем фильмам.

In [56]:
average_rating = data.agg(f.sum("rating").alias("sum_rating"),
                          f.count("item_id").alias("count_rating"),
                         (f.sum("rating")/f.count("item_id")).alias("avr_rating"))

In [57]:
average_rating.show()

+----------+------------+----------+
|sum_rating|count_rating|avr_rating|
+----------+------------+----------+
|    352986|      100000|   3.52986|
+----------+------------+----------+



In [58]:
round(3.52986,4)

3.5299

In [91]:
data_user.where("user_id = 758").show(3)

+-------+----------+------------+-----------------+
|user_id|sum_rating|count_rating|       avr_rating|
+-------+----------+------------+-----------------+
|    758|      1384|         357|3.876750700280112|
+-------+----------+------------+-----------------+



2) Базовый предиктор для каждого пользователя (суммирование по фильмам, оцененным данным пользователем). Здесь Ia — множество фильмов, по которым у пользователя есть рейтинги, а |Ia| — их количество.

In [94]:
query4 = """
select d.user_id,
       sum(rating - 3.52986)/(count(d.item_id) + 10) as bu
from   data_table d 
group by d.user_id
"""

In [95]:
user_bu = spark.sql(query4)

проверим для нашего пользователя

In [96]:
user_bu.where("user_id = 758").show(2)

+-------+-------------------+
|user_id|                 bu|
+-------+-------------------+
|    758|0.33743863760217984|
+-------+-------------------+



In [92]:
data.where("user_id = 758").count()

357

In [75]:
(1384 - 3.52986*357)/(357+10)

0.33743863760217974

In [97]:
user_bu.registerTempTable("user_bu_table") 

3.Базовый предиктор для каждого фильма (суммирование по пользователям, поставившим оценку данному фильму). Здесь Ui — множество пользователей, которые оценили данный фильм, а |Ui| — их количество.

In [98]:
query5 = """
select d.item_id,
       sum(rating - u.bu - 3.52986)/(count(d.user_id) + 25) as bi
from   data_table d join user_bu_table u on u.user_id = d.user_id
group by d.item_id
"""

In [99]:
item_bi = spark.sql(query5)

In [100]:
item_bi.where("item_id = 1").show(3)

+-------+-----------------+
|item_id|               bi|
+-------+-----------------+
|      1|0.292924886016311|
+-------+-----------------+



In [101]:
item_bi.registerTempTable("item_bi_table") 

In [102]:
item_bi.take(4)

[Row(item_id=496, bi=Decimal('0.430520332931035')),
 Row(item_id=463, bi=Decimal('0.163228208578692')),
 Row(item_id=471, bi=Decimal('0.054588950305639')),
 Row(item_id=148, bi=Decimal('-0.251341569613545'))]

In [103]:
item_bi.count()

1682

4.Базовый предиктор для каждого пользователя и каждого фильма:
(количество пользователей * количество фильмов)

In [104]:
users.registerTempTable("users_table")

In [105]:
item.registerTempTable("item_table")

In [106]:
query5_2 = """
select i.item_id, u.user_id
from   users_table u 
cross join item_table i
"""

In [107]:
table_cross_all = spark.sql(query5_2).repartition(4).cache()

In [108]:
table_cross_all.count()/users.count()

1682.0

In [109]:
table_cross_all.count()/item.count()

943.0

In [110]:
table_cross_all.registerTempTable("table_cross_all_table")

Заполним нашу крос таблицу (количество пользователей * количество фильмов) данными

In [111]:
query5_1 = """
SELECT ct.item_id, ct.user_id, 
       3.52986 as mu,
       u.bu,
       i.bi, 
       (3.52986+nvl(u.bu,0)+nvl(i.bi,0)) as bui_base,
       nvl(t.rating,0) as rating
FROM   table_cross_all_table ct
join   item_bi_table i on i.item_id = ct.item_id
join   user_bu_table u on u.user_id = ct.user_id
left outer join data_table t on ct.item_id = t.item_id and ct.user_id = t.user_id
"""       

In [112]:
bui_cross = spark.sql(query5_1).repartition(4).cache()

In [114]:
bui_cross.count()/users.count()

1682.0

In [120]:
bui_cross.registerTempTable("bui_table")

In [116]:
bui_cross.where("user_id = 758 and item_id = 135").show(4)

+-------+-------+-------+-------------------+-----------------+----------------+------+
|item_id|user_id|     mu|                 bu|               bi|        bui_base|rating|
+-------+-------+-------+-------------------+-----------------+----------------+------+
|    135|    758|3.52986|0.33743863760217984|0.328434813076320|4.19573345067850|     5|
+-------+-------+-------+-------------------+-----------------+----------------+------+



In [117]:
user_bu.where("user_id = 758").show()

+-------+-------------------+
|user_id|                 bu|
+-------+-------------------+
|    758|0.33743863760217984|
+-------+-------------------+



In [118]:
item_bi.where("item_id = 135").show()

+-------+-----------------+
|item_id|               bi|
+-------+-----------------+
|    135|0.328434813076320|
+-------+-----------------+



In [119]:
(3.52986+0.33743863760217984+0.328434813076320)

4.1957334506785

In [121]:
bui_cross.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- mu: decimal(6,5) (nullable = false)
 |-- bu: decimal(38,17) (nullable = true)
 |-- bi: decimal(38,15) (nullable = true)
 |-- bui_base: decimal(38,14) (nullable = true)
 |-- rating: integer (nullable = false)



# Часть 4. Item-item CF:

Создадим клона нашей таблицы, так как сама с собой в spark не работала

In [122]:
query66_clon = """
SELECT item_id as item_i,
       user_id as user_i,
       mu as mu_i,
       bu as bu_i,
       bi as bi_i,
       bui_base as bui_base_i,
       rating as rating_i
FROM   bui_table b
"""       

In [130]:
bui_i = spark.sql(query66_clon).repartition(4).cache()

In [131]:
bui_i.registerTempTable("bui_i_table")

2. Найдите попарные меры близости (косинус) для всех фильмов, используя очищенные оценки из пункта 4.1. Суммирование идет по всем пользователям. 

Считаем знаменатель. По всем пользователям, которые поставили рейтинг данному фильму!

In [153]:
query_norm = """
SELECT b.item_id,
       sqrt(sum((b.rating-b.bui_base)*(b.rating-b.bui_base))) as norm
FROM   bui_table b
where  b.rating <> 0
group by b.item_id
"""       

In [154]:
item_norm = spark.sql(query_norm).repartition(4).cache()

In [155]:
item_norm.where("item_id = 1")\
    .show()

+-------+-----------------+
|item_id|             norm|
+-------+-----------------+
|      1|17.95777926136748|
+-------+-----------------+



In [126]:
item_norm.count()

1682

In [127]:
item_norm.registerTempTable("item_norm")

1. Вычесть из всех рейтингов rui базовый предиктор bui из пункта 3.4 (для всей таблицы рейтингов). Считаем числитель по пользователям, которые оценили оба сравниваемых фильма. Т.е. числитель считается по пересечению множества пользователей у данного фильма.
2. Найдите попарные меры близости (косинус) для всех фильмов, используя очищенные оценки из пункта 4.1. Суммирование идет по всем пользователям (для знаменателя - норма по всем оценкам пользователей, оценившии фильм i). 

Мера близости считается между фильмами, по которым есть пользовательская оценка. 

In [156]:
query66 = """
SELECT b.item_id as item_i, bb.item_i as item_j,
       sum((b.rating - b.bui_base)*(bb.rating_i - bb.bui_base_i))/
       (min(in1.norm)*min(in2.norm)) as sij,
       min(in1.norm) as norm_i, min(in2.norm) as norm_j
FROM   bui_table b join item_norm in1 on in1.item_id = b.item_id
join   bui_i_table bb on b.user_id = bb.user_i
join   item_norm in2 on in2.item_id = bb.item_i
where  b.rating <> 0
and    bb.rating_i <> 0
group by b.item_id, bb.item_i
"""       

In [157]:
sij_proba = spark.sql(query66).repartition(4).cache()

In [158]:
sij_proba.select('item_i').distinct().count()

1682

In [159]:
sij_proba.where("item_i =1 and item_j = 404 ").show()

+------+------+------------------+-----------------+-----------------+
|item_i|item_j|               sij|           norm_i|           norm_j|
+------+------+------------------+-----------------+-----------------+
|     1|   404|0.0259744361002555|17.95777926136748|8.315947330280537|
+------+------+------------------+-----------------+-----------------+



In [160]:
sij_proba.registerTempTable("sij_table") 

3. Для каждого фильма, по которому у данного пользователя не стоит рейтинг, найдите:

- [a] 30 ближайших фильмов-соседей для этого фильма (среди всех фильмов, а не фильмов, оценённых пользователем).

item_i - это фильмы, оценку которых мы будем предсказать

item_j - фильм-сосед, не зависит оценил ли наш пользователь его или нет

rank - по убывания меры близости

rank_pos - по возрастанию меры близости (отрицательные будут выше)

In [141]:
query7 = """
SELECT sij.item_i, sij.item_j, sij.sij,
       ROW_NUMBER() OVER (PARTITION BY sij.item_i ORDER BY sij.sij DESC, sij.item_j ASC) rank,
       ROW_NUMBER() OVER (PARTITION BY sij.item_i ORDER BY sij.sij, sij.item_j ASC) rank_pos
FROM   sij_table sij
where  not exists (select item_id from item_758_table ip where ip.item_id = sij.item_i)
and    sij.item_i<>sij.item_j
"""       

In [142]:
neib_item = spark.sql(query7).repartition(4).cache()

In [143]:
neib_item.registerTempTable("neib_item_table") 

In [144]:
neib_item.where("item_i = 1").count()

1593

In [145]:
neib_item.orderBy(f.asc("sij")).show(10)

+------+------+-------------------+----+--------+
|item_i|item_j|                sij|rank|rank_pos|
+------+------+-------------------+----+--------+
|   814|   857|-1.0000094294401458| 635|       1|
|   857|   814|-1.0000094294401458| 635|       1|
|  1666|  1668|-1.0000074318388779| 230|       2|
|  1666|  1667|-1.0000074318388779| 229|       1|
|  1668|  1669|-1.0000074318388779| 231|       4|
|  1665|  1667|-1.0000074318388779| 229|       1|
|  1669|  1668|-1.0000074318388779| 230|       2|
|  1670|  1663|-1.0000074318388779| 228|       1|
|  1667|  1666|-1.0000074318388779| 230|       3|
|  1665|  1670|-1.0000074318388779| 231|       3|
+------+------+-------------------+----+--------+
only showing top 10 rows



In [334]:
neib_item.printSchema()

root
 |-- item_i: integer (nullable = true)
 |-- item_j: integer (nullable = true)
 |-- sij: double (nullable = true)
 |-- rank: integer (nullable = true)



In [146]:
neib_item_distinct = neib_item[["item_i"]].distinct() 

In [147]:
neib_item_distinct.count()+item_758.count()

1682

[b] прогноз оценки пользователя по формуле (базовый предиктор из пункта 4, часть 3). 
Здесь S(i)- множество фильмов-соседей для фильма i, по которым у данного пользователя есть оценка.

Самые близкие соседи - самая высокая мера близости, топ 30

In [148]:
query77 = """
SELECT t.item_i, t.item_j, t.sij, t.rank
FROM   neib_item_table t
where  t.rank < 31
order  by 1, 3 desc, 2
"""       

In [162]:
neib_item_30 = spark.sql(query77).repartition(4).cache()

In [205]:
neib_item_30.where("sij< 0").orderBy("rank").show(3)

+------+------+--------------------+----+
|item_i|item_j|                 sij|rank|
+------+------+--------------------+----+
|  1653|   242|-0.00631707020451...|  21|
|  1653|  1007|-0.00870405330101...|  22|
|  1653|   258|-0.04182908819864019|  23|
+------+------+--------------------+----+
only showing top 3 rows



In [151]:
neib_item_30.registerTempTable("neib_item30_table") 

In [338]:
bui_cross.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- mu: decimal(6,5) (nullable = false)
 |-- bu: decimal(38,17) (nullable = true)
 |-- bi: decimal(38,15) (nullable = true)
 |-- bui_base: decimal(38,14) (nullable = true)
 |-- rating: integer (nullable = false)



In [164]:
bui_cross.where("user_id = 758").show()

+-------+-------+-------+-------------------+------------------+----------------+------+
|item_id|user_id|     mu|                 bu|                bi|        bui_base|rating|
+-------+-------+-------+-------------------+------------------+----------------+------+
|   1265|    758|3.52986|0.33743863760217984|-0.133829052253318|3.73346958534886|     0|
|    939|    758|3.52986|0.33743863760217984| 0.059006633437362|3.92630527103954|     0|
|     81|    758|3.52986|0.33743863760217984|-0.000435142700680|3.86686349490150|     5|
|    460|    758|3.52986|0.33743863760217984|-0.201399487439182|3.66589915016300|     0|
|     22|    758|3.52986|0.33743863760217984| 0.516160817694467|4.38345945529665|     0|
|    209|    758|3.52986|0.33743863760217984| 0.273243894625916|4.14054253222810|     5|
|    995|    758|3.52986|0.33743863760217984|-0.109153437387883|3.75814520021430|     0|
|    372|    758|3.52986|0.33743863760217984|-0.001779340693821|3.86551929690836|     0|
|   1586|    758|3.52

- [b] прогноз оценки пользователя по формуле (базовый предиктор из пункта 3.4 - берется для предсказывающего фильма + пользователя). Здесь S(i)- множество фильмов-соседей для фильма i, по которым у данного пользователя есть оценка.

In [167]:
query888 = """
SELECT i.item_i,
       bi.bui_base + sum(i.sij*(bj.rating_i - bj.bui_base_i))/sum(abs(i.sij)) as rating_new
FROM   neib_item30_table i 
join   bui_table bi on bi.item_id = i.item_i and bi.user_id = 758
join   bui_i_table bj on bj.item_i = i.item_j and bj.user_i = 758 and bj.rating_i <> 0
group  by i.item_i, bi.bui_base
"""       

5. При подсчете прогноза по формуле из пункта 4.3 отфильтруйте всех соседей с отрицательной близостью. Т.е. возьмем только положительную меру близости, откиним фильмы с противоположным отношением.

In [199]:
query888_pos = """
SELECT i.item_i,
       bi.bui_base + sum(i.sij*(bj.rating_i - bj.bui_base_i))/sum(abs(i.sij)) as rating_new
FROM   neib_item30_table i 
join   bui_table bi on bi.item_id = i.item_i and bi.user_id = 758
join   bui_i_table bj on bj.item_i = i.item_j and bj.user_i = 758 and bj.rating_i <> 0
where  i.sij > 0
group  by i.item_i, bi.bui_base
"""       

In [248]:
neib_item_30.where("item_i = 22").count()

30

In [168]:
neib_item30_predict = spark.sql(query888).repartition(4).cache()

In [169]:
neib_item30_predict.groupBy("item_i").count().show(3)

+------+-----+
|item_i|count|
+------+-----+
|   463|    1|
|  1591|    1|
|   833|    1|
+------+-----+
only showing top 3 rows



In [200]:
neib_item30_predict_pos = spark.sql(query888_pos).repartition(4).cache()

In [201]:
neib_item30_predict_pos.count()

1024

In [206]:
neib_item30_predict_pos.orderBy(f.desc("rating_new")).show(10)

+------+-----------------+
|item_i|       rating_new|
+------+-----------------+
|  1416|5.396102257568391|
|   360| 5.27863280760301|
|  1018| 5.24635181221795|
|   989| 5.24579614449314|
|  1121| 5.10301310973268|
|   190| 5.08969199267952|
|  1011| 5.08087777501547|
|  1097| 5.04777486875467|
|   878|  5.0376502268368|
|  1456| 5.02231890348717|
+------+-----------------+
only showing top 10 rows



In [207]:
neib_item30_predict_pos.orderBy(f.desc("rating_new")).show(10)

+------+-----------------+
|item_i|       rating_new|
+------+-----------------+
|  1416|5.396102257568391|
|   360| 5.27863280760301|
|  1018| 5.24635181221795|
|   989| 5.24579614449314|
|  1121| 5.10301310973268|
|   190| 5.08969199267952|
|  1011| 5.08087777501547|
|  1097| 5.04777486875467|
|   878|  5.0376502268368|
|  1456| 5.02231890348717|
+------+-----------------+
only showing top 10 rows



In [171]:
predicators_top10 = [x['item_i'] for x in neib_item30_predict.orderBy(f.desc("rating_new")).take(10)]

In [172]:
predicators_top10

[1416, 360, 1018, 989, 1121, 190, 1011, 1097, 878, 1456]

In [202]:
predicators_positive_top10 = [x['item_i'] for x in neib_item30_predict_pos.orderBy(f.desc("rating_new")).take(10)]

In [208]:
predicators_positive_top10

[1416, 360, 1018, 989, 1121, 190, 1011, 1097, 878, 1456]

# Результат

In [203]:
dict_out_new = {
              "average_rating": 3.5299,
              "predicators_positive_top10": predicators_positive_top10,
              "predicators_top10": predicators_top10
             }

In [204]:
import json
with open('lab08s.json', 'w') as fout:
    fout.write(json.dumps(dict_out_new))

In [209]:
spark.stop()